In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
vectors_df = pd.read_csv('vectors.txt', delim_whitespace=True, header=None)
parsed_col = []
for keyword in list(vectors_df[0]):
    keyword = str(keyword)
    keyword = keyword.replace('_', ' ')
    keyword = keyword.replace('’', "'")
    parsed_col.append(keyword)
vectors_df[0] = parsed_col
vectors_df

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,300
0,singapore,0.483909,0.906052,-0.081405,-0.402945,-0.047337,-0.211383,0.105737,-0.323959,0.510133,...,-0.014059,0.151803,0.018653,0.178135,-0.288335,0.176272,0.748743,0.934583,-0.303212,0.107761
1,place,0.983583,0.080516,0.307405,-0.478417,-0.265591,0.097866,0.006518,-0.024012,0.867993,...,-0.764314,-0.474266,0.451464,0.648060,0.041233,0.448002,0.597665,0.580291,-0.613324,0.685953
2,gardens,0.631589,0.493373,0.344579,0.329813,-0.687817,-0.231521,-0.636084,0.556939,-0.330538,...,-0.464227,-0.003085,0.771097,-0.186565,-0.653929,-0.132298,0.597235,0.534219,0.756995,-0.097779
3,time,1.287348,0.745675,0.804061,-0.626885,0.316215,-0.374367,0.264915,-0.753390,-0.060295,...,0.337025,-1.014939,-0.187284,0.281302,-0.819577,-0.510459,0.313020,0.121017,-0.042820,-0.470455
4,night,0.571266,0.875816,-0.591916,-0.490323,-0.734806,0.068555,0.439737,0.373520,0.375961,...,-0.240506,-0.104857,-0.364867,-0.497946,-0.469053,-0.853215,0.371460,-0.077505,0.404010,0.513231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20644,zoo site,-0.075530,-0.020636,0.014292,0.036398,0.023679,0.046482,0.109715,0.034369,-0.073305,...,0.008565,0.019394,0.001102,-0.046221,0.021223,-0.033101,-0.035547,-0.036359,-0.025521,-0.030565
20645,zoo tour,-0.085650,-0.093741,0.015703,0.000577,-0.044756,-0.019462,0.033156,0.043889,-0.016135,...,-0.039359,-0.024103,-0.037887,-0.031642,0.035126,0.030659,0.034387,-0.029270,-0.023102,0.028840
20646,zoo yesterday,-0.000614,-0.019895,0.037936,0.041266,0.010509,-0.008908,-0.055139,0.003879,0.004546,...,0.015647,-0.014379,0.007880,0.053511,-0.051227,0.013682,-0.028846,0.012478,-0.002388,-0.062719
20647,zoological gardens,-0.085428,-0.061564,-0.039814,0.031094,-0.026854,-0.031232,0.040986,0.042299,-0.005840,...,0.024237,0.017969,-0.043686,-0.045863,0.056995,-0.008930,-0.053685,-0.084168,0.011695,0.033877


In [3]:
keywords_df = pd.read_excel('20200304_Aspects_Annotation.xlsx')
keywords_df = keywords_df[keywords_df['count'] >= 5]
keywords_df

,TEXT,count,SENTIMENT_KEYWORD_AGG_INDEX,cumulative,ASPECT_CATEGORY_NAME,ASPECT_CATEGORY,REQUIRE_CONTEXT
0,singapore,26872,1,26872,NaN,NaN,1.0
1,place,16418,2,43290,Sightseeing/ Place,NaN,NaN
2,gardens,15431,3,58721,Sightseeing/ Place,NaN,NaN
3,time,11895,4,70616,NaN,NaN,1.0
4,night,11791,5,82407,NaN,NaN,1.0
...,...,...,...,...,...,...,...
20642,zoo prices,5,20643,1105318,NaN,NaN,NaN
20643,zoo site,5,20644,1105323,NaN,NaN,NaN
20644,zoo tour,5,20645,1105328,NaN,NaN,NaN
20645,zoo yesterday,5,20646,1105333,NaN,NaN,NaN


In [4]:
parsed_col_2 = []
for keyword in list(keywords_df['TEXT']):
    keyword = str(keyword)
    keyword = keyword.replace('â€™', "'")
    keyword = keyword.replace('ã©', 'é')
    parsed_col_2.append(keyword)
keywords_df['TEXT'] = parsed_col_2

In [5]:
vectors_keywords_list = list(vectors_df[0])
ibm_keywords_list = list(keywords_df['TEXT'])

In [6]:
## Index of GloVe keywords not in IBM keyword list
for keyword in vectors_keywords_list:
    if keyword not in ibm_keywords_list:
        print(vectors_keywords_list.index(keyword))

11764
14975
15268
17312
17313
20648


In [7]:
## Index of IBM keywords not in GloVe keyword list
for keyword in ibm_keywords_list:
    if keyword not in vectors_keywords_list:
        print(ibm_keywords_list.index(keyword))

11765
15013
17371
17442


In [8]:
matrix = vectors_df.iloc[:, 1:].values
matrix.shape

(20649, 300)

In [9]:
def get_centroid(matrix):
    return np.mean(matrix, axis=0)

In [10]:
## mahatten, euclidean, cosine similarity, correlation

def get_manhattan(a, b):
    return np.abs(a - b).sum()

def get_euclidean(a, b):
    return np.linalg.norm(a-b)

def get_cos_sim(a, b):
    return np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))

def get_corr(a, b):
    return np.correlate(a, b)[0]

In [11]:
aspect_categories = list(set(keywords_df.ASPECT_CATEGORY_NAME))
aspect_categories = aspect_categories[1:]  # removing the nan category
aspect_categories.sort()
aspect_categories

['Activities',
 'Environment and Ambience',
 'Experience',
 'F&B',
 'Facilities',
 'Family/Children family/ Friends',
 'Price',
 'Service',
 'Shopping',
 'Sightseeing/ Place',
 'Transportation and Accessibility']

In [12]:
keywords_df.iloc[:400,:]

,TEXT,count,SENTIMENT_KEYWORD_AGG_INDEX,cumulative,ASPECT_CATEGORY_NAME,ASPECT_CATEGORY,REQUIRE_CONTEXT
0,singapore,26872,1,26872,NaN,NaN,1.0
1,place,16418,2,43290,Sightseeing/ Place,NaN,NaN
2,gardens,15431,3,58721,Sightseeing/ Place,NaN,NaN
3,time,11895,4,70616,NaN,NaN,1.0
4,night,11791,5,82407,NaN,NaN,1.0
...,...,...,...,...,...,...,...
395,shoes,391,396,592554,NaN,NaN,1.0
396,car,390,397,592944,Transportation and Accessibility,NaN,NaN
397,venue,390,398,593334,Transportation and Accessibility,NaN,1.0
398,couples,388,399,593722,Family/Children family/ Friends,NaN,NaN


In [13]:
train_size = 400
annotated_size = 500

df_train = keywords_df.iloc[:train_size, :]
no_of_categories = len(aspect_categories)
centroids = np.zeros((no_of_categories, 300))

for i in range(no_of_categories):
    indices = df_train[df_train.ASPECT_CATEGORY_NAME == aspect_categories[i]].index
    centroids[i] = get_centroid(matrix[indices])
    
centroids

array([[ 0.37979293,  0.28584743,  0.12800629, ...,  0.30231271,
        -0.155568  ,  0.01538829],
       [ 0.26015037,  0.20712421, -0.17060329, ...,  0.16617029,
         0.19971004,  0.11724054],
       [ 0.10898367,  0.16351621, -0.01123192, ...,  0.08730217,
         0.15257458,  0.08237075],
       ...,
       [ 0.646062  ,  0.06382786,  0.521565  , ...,  0.212083  ,
        -0.09034086,  0.116248  ],
       [ 0.27963617,  0.19527255, -0.07924848, ...,  0.15581457,
         0.07552268,  0.15543688],
       [ 0.30724739,  0.13110241,  0.04729133, ...,  0.2353499 ,
         0.07867645,  0.16170339]])

In [14]:
manhattan_distances = np.zeros((matrix.shape[0], no_of_categories))
euclidean_distances = np.zeros((matrix.shape[0], no_of_categories))
cos_sim = np.zeros((matrix.shape[0], no_of_categories))
corr = np.zeros((matrix.shape[0], no_of_categories))

def get_distances(centroids, matrix):
    for i in range(matrix.shape[0]):
        for j in range(no_of_categories):
            manhattan_distances[i, j] = get_manhattan(matrix[i], centroids[j])
            
    for i in range(matrix.shape[0]):
        for j in range(no_of_categories):
            euclidean_distances[i, j] = get_euclidean(matrix[i], centroids[j])
            
    for i in range(matrix.shape[0]):
        for j in range(no_of_categories):
            cos_sim[i, j] = get_cos_sim(matrix[i], centroids[j])
            
    for i in range(matrix.shape[0]):
        for j in range(no_of_categories):
            correlations[i, j] = get_corr(matrix[i], centroids[j])

In [15]:
%%time 
get_distances(centroids, matrix)

NameError: name 'correlations' is not defined

In [16]:
def parse_to_df(matrix):
    final_df = pd.DataFrame(data=matrix)
    final_df['KEYWORD'] = vectors_df[0]
    cols = ['KEYWORD'] + [i for i in range(no_of_categories)]
    colnames = ['KEYWORD'] + [i for i in range(1, no_of_categories+1)]
    
    final_df = final_df.loc[:, cols]
    final_df.columns = colnames
    return final_df

manhattan_df = parse_to_df(manhattan_distances)
euclidean_df = parse_to_df(euclidean_distances)
cos_sim_df = parse_to_df(cos_sim)
corr_df = parse_to_df(corr)

euclidean_df.head()

,KEYWORD,1,2,3,4,5,6,7,8,9,10,11
0,singapore,7.169875,7.229749,7.430225,7.399440,7.901524,7.355432,7.323339,7.416677,7.866855,7.015753,7.343704
1,place,8.202251,8.127702,8.112960,8.282930,8.599186,8.013188,8.257445,8.571232,8.274454,7.830904,8.096079
2,gardens,8.704792,8.236179,8.415784,8.700394,8.697481,8.474440,8.474535,9.025886,9.239223,8.062364,8.392451
3,time,8.088859,8.336515,8.492066,8.588680,8.755009,8.329051,8.478879,8.121401,8.985488,8.176048,8.282589
4,night,8.676596,8.484446,8.528526,8.484338,8.579590,8.631850,8.651631,9.201082,9.037385,8.240517,8.416943


In [17]:
def get_closest_index(row):
    dictionary = dict(zip(row, list(range(1, no_of_categories+1))))
    return dictionary[min(row)]

aspect_categories_numbered = dict(zip(range(1, 12), aspect_categories))
aspect_categories_numbered

{1: 'Activities',
 2: 'Environment and Ambience',
 3: 'Experience',
 4: 'F&B',
 5: 'Facilities',
 6: 'Family/Children family/ Friends',
 7: 'Price',
 8: 'Service',
 9: 'Shopping',
 10: 'Sightseeing/ Place',
 11: 'Transportation and Accessibility'}

In [18]:
%%time

for df in [manhattan_df, euclidean_df, cos_sim_df, corr_df]:
    df['CLOSEST'] = df.iloc[:,1:no_of_categories+1].apply(
        get_closest_index, 
        axis=1
    )
    df['ASPECT_CATEGORY_NAME'] = [aspect_categories_numbered[x] for x in df['CLOSEST']]
    
euclidean_df.head()

CPU times: user 1.93 s, sys: 3.94 ms, total: 1.94 s
Wall time: 1.94 s


,KEYWORD,1,2,3,4,5,6,7,8,9,10,11,CLOSEST,ASPECT_CATEGORY_NAME
0,singapore,7.169875,7.229749,7.430225,7.399440,7.901524,7.355432,7.323339,7.416677,7.866855,7.015753,7.343704,10,Sightseeing/ Place
1,place,8.202251,8.127702,8.112960,8.282930,8.599186,8.013188,8.257445,8.571232,8.274454,7.830904,8.096079,10,Sightseeing/ Place
2,gardens,8.704792,8.236179,8.415784,8.700394,8.697481,8.474440,8.474535,9.025886,9.239223,8.062364,8.392451,10,Sightseeing/ Place
3,time,8.088859,8.336515,8.492066,8.588680,8.755009,8.329051,8.478879,8.121401,8.985488,8.176048,8.282589,1,Activities
4,night,8.676596,8.484446,8.528526,8.484338,8.579590,8.631850,8.651631,9.201082,9.037385,8.240517,8.416943,10,Sightseeing/ Place


In [19]:
def get_test_indices(df):
    temp_df = df.iloc[train_size:annotated_size, :]
    return temp_df[temp_df.ASPECT_CATEGORY_NAME.isin(aspect_categories)].index

test_indices = get_test_indices(keywords_df)
test_indices

Int64Index([401, 402, 404, 405, 406, 407, 408, 409, 410, 412, 413, 414, 415,
            416, 418, 420, 423, 424, 425, 428, 429, 430, 431, 432, 433, 434,
            435, 436, 437, 438, 441, 442, 446, 447, 449, 450, 451, 457, 459,
            463, 464, 465, 470, 471, 472, 473, 476, 477, 478, 481, 482, 484,
            485, 486, 487, 490, 491, 492, 493, 495, 498, 499],
           dtype='int64')

In [20]:
def get_accuracy(model):
    sum = (model.ASPECT_CATEGORY_NAME[test_indices] == keywords_df.ASPECT_CATEGORY_NAME[test_indices]).sum()
    print("Accuracy: {}".format(sum/len(test_indices)))

for df in [manhattan_df, euclidean_df, cos_sim_df, corr_df]:
    get_accuracy(df)

Accuracy: 0.46774193548387094
Accuracy: 0.46774193548387094
Accuracy: 0.0
Accuracy: 0.1935483870967742
